In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
weekend_Jul_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in weekend_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

25920

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 2921.6936757 ,  1542.79974226],
        [ 2436.9682986 ,  3499.63661529],
        [ 3759.05641009,  3641.25236258],
        [ 5627.25643956,  5230.57258943],
        [ 4826.52575298,  4745.26914424],
        [ 1627.79604823,  2600.11964125],
        [ 3207.73085053,  3826.71238546],
        [ 3870.54522946,  3135.52301083],
        [ 4429.79428332,  3706.90359387],
        [ 4473.6744229 ,  5195.46482137],
        [ 3155.67796838,  4790.42706306],
        [ 4228.81344028,  4452.26408732],
        [ 3462.34304237,  2385.23806519],
        [ 4726.48323431,  2832.50174502],
        [ 1421.72349669,  2235.68006175],
        [ 1794.43681305,  1716.69206256],
        [ 3494.697493  ,  2827.83781317],
        [ 2881.37535551,  2861.20278105],
        [ 3088.83581163,  3041.22053895],
        [ 2753.34902367,  3533.88854191],
        [ 2906.85168423,  2620.25695734],
        [ 1339.67495088,  1544.25826859],
        [ 3433.22243065,  3498.46490127],
        [ 3760.23807314,  3227.719

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 1079)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e-03, 9e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.86337034e+06  0.00000000e+00  6.73e+03 1.23e+02  9.52e+05     0s
   1   1.42811618e+06 -2.20141195e+06  9.81e+02 1.37e+02  2.01e+05     0s
   2   6.71146001e+05 -1.02036764e+06  6.92e+00 8.41e-01  3.08e+04     0s
   3   3.63537562e+04 -2.10860988e+05  6.92e-06 5.68e-14  4.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[8.558171540147492e-10,
 4349.255567633862,
 9.394052226310688e-10,
 85.7283623605092,
 950.1025323179944,
 2.0307428402739045e-06,
 2.488496877574486e-07,
 0.001593963498688072,
 170.73070929072014,
 2567.735304488179,
 82.76478473515785,
 144.7645079873716,
 2.0306115580034426e-06,
 2.042232970802594e-06,
 2960.3993298589803,
 467.5658845868958,
 99.16728842989606,
 84.41469650618859,
 1230.5953665931686,
 3422.519184582577,
 2.031339278165107e-06,
 1711.2289249617343,
 861.1408969004533,
 255.677927255403,
 1483.4628690182383,
 678.3558402311587,
 1.1643793742317272e-06,
 2471.142570330282,
 0.5506826589365366,
 0.5983043112890813,
 400.2028443495647,
 0.3065554081292798,
 1495.9771369596967,
 1.1527135425213862e-06,
 1.1532952832843544e-06,
 1301.8005780117587,
 465.0174211506828,
 4.801140078681702e-05,
 1374.936142774076,
 176.25560584994236,
 183.55653976043666,
 247.54859391859208,
 1.5045207929602361e-06,
 1.504502462945418e-06,
 2721.0030483192327,
 65.8788748976628,
 673.172